In [ ]:
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets
import numpy
import matplotlib.pyplot

In [ ]:
x = tensorflow.keras.layers.Input(shape=(784), name="encoder_input")

In [ ]:
encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=300, name="encoder_dense_1")(x)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

In [ ]:
encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=2, name="encoder_dense_2")(encoder_activ_layer1)
encoder_output = tensorflow.keras.layers.LeakyReLU(name="encoder_output")(encoder_dense_layer2)

In [ ]:
encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

In [ ]:
decoder_input = tensorflow.keras.layers.Input(shape=(2), name="decoder_input")

In [ ]:
decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=300, name="decoder_dense_1")(decoder_input)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

In [ ]:
decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=784, name="decoder_dense_2")(decoder_activ_layer1)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_output")(decoder_dense_layer2)

In [ ]:
decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

In [ ]:
ae_input = tensorflow.keras.layers.Input(shape=(784), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

In [ ]:
ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
ae.summary()

In [ ]:
def rmse(y_true, y_predict):
  return tensorflow.keras.backend.mean(tensorflow.keras.backend.square(y_true-y_predict))

In [ ]:
ae.compile(loss="mse", optimizer=tensorflow.keras.optimizers.Adam(lr=0.0005))

In [ ]:
(x_train_orig, y_train), (x_test_orig, y_test) = tensorflow.keras.datasets.mnist.load_data()
x_train_orig = x_train_orig.astype("float32") / 255.0
x_test_orig = x_test_orig.astype("float32") / 255.0

In [ ]:
x_train = numpy.reshape(x_train_orig, newshape=(x_train_orig.shape[0], numpy.prod(x_train_orig.shape[1:])))
x_test = numpy.reshape(x_test_orig, newshape=(x_test_orig.shape[0], numpy.prod(x_test_orig.shape[1:])))

In [ ]:
ae.fit(x_train, x_train, epochs=20, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

In [ ]:
encoded_images = encoder.predict(x_train)
decoded_images = decoder.predict(encoded_images)
decoded_images_orig = numpy.reshape(decoded_images, newshape=(decoded_images.shape[0], 28, 28))

In [ ]:
num_images_to_show = 5
for im_ind in range(num_images_to_show):
  plot_ind = im_ind*2 + 1
  rand_ind = numpy.random.randint(low=0, high=x_train.shape[0])
  matplotlib.pyplot.subplot(num_images_to_show, 2, plot_ind)
  matplotlib.pyplot.imshow(x_train_orig[rand_ind, :, :], cmap="gray")
  matplotlib.pyplot.subplot(num_images_to_show, 2, plot_ind+1)
  matplotlib.pyplot.imshow(decoded_images_orig[rand_ind, :, :], cmap="gray")

In [ ]:
matplotlib.pyplot.figure()
matplotlib.pyplot.scatter(encoded_images[:, 0], encoded_images[:, 1], c=y_train)
matplotlib.pyplot.colorbar()